# Lesson A1: Lifetimes & Advanced Traits

**Duration**: 150-165 minutes  
**Stage**: Advanced (Mastery)  
**Prerequisites**: All Intermediate lessons (I1-I5)

---

## 📋 What You'll Learn

This lesson dives deep into Rust's lifetime system and advanced trait patterns. You'll master lifetime annotations, understand elision rules, and explore sophisticated trait techniques like associated types and higher-ranked trait bounds.

**Why this matters**: Lifetimes are what make Rust's memory safety guarantees possible without garbage collection. Advanced trait patterns enable powerful abstractions used in production Rust code, from async runtimes to database ORMs.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Understand lifetime annotations and elision rules
2. Apply lifetimes to references and structs
3. Use higher-ranked trait bounds
4. Implement advanced trait patterns
5. Work with associated types and type families
6. Master trait object lifetimes


In [ ]:
// Basic lifetime examples

// Function that returns the longer of two string slices
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

// Function with multiple lifetime parameters
fn first_word<'a, 'b>(x: &'a str, _y: &'b str) -> &'a str {
    x.split_whitespace().next().unwrap_or("")
}

// Struct with lifetime parameter
#[derive(Debug)]
struct ImportantExcerpt<'a> {
    part: &'a str,
}

impl<'a> ImportantExcerpt<'a> {
    // Method with lifetime elision
    fn level(&self) -> i32 {
        3
    }
    
    // Method that returns a reference with same lifetime as self
    fn announce_and_return_part(&self, announcement: &str) -> &str {
        println!("Attention please: {}", announcement);
        self.part
    }
}

fn basic_lifetimes_demo() {
    println!("=== Basic Lifetimes Demo ===");
    
    // Simple lifetime example
    let string1 = String::from("abcd");
    let string2 = "xyz";
    
    let result = longest(string1.as_str(), string2);
    println!("The longest string is: '{}'", result);
    
    // Multiple lifetimes
    let sentence = "Hello world from Rust";
    let other = "Goodbye";
    let first = first_word(sentence, other);
    println!("First word: '{}'", first);
    
    // Struct with lifetime
    let novel = String::from("Call me Ishmael. Some years ago...");
    let first_sentence = novel.split('.').next().expect("Could not find a '.'");
    let excerpt = ImportantExcerpt {
        part: first_sentence,
    };
    
    println!("Excerpt: {:?}", excerpt);
    println!("Level: {}", excerpt.level());
    
    let announcement = "Today we're reading a classic";
    let part = excerpt.announce_and_return_part(announcement);
    println!("Returned part: '{}'", part);
}

basic_lifetimes_demo();

In [ ]:
use std::fmt::Display;

// Advanced lifetime patterns

// Function with lifetime bounds
fn longest_with_an_announcement<'a, T>(
    x: &'a str,
    y: &'a str,
    ann: T,
) -> &'a str
where
    T: Display,
{
    println!("Announcement! {}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

// Struct with multiple lifetime parameters
#[derive(Debug)]
struct Context<'s, 'c> {
    string_data: &'s str,
    config_data: &'c str,
}

impl<'s, 'c> Context<'s, 'c> {
    fn new(string_data: &'s str, config_data: &'c str) -> Self {
        Context {
            string_data,
            config_data,
        }
    }
    
    // Method that returns reference with specific lifetime
    fn get_string_data(&self) -> &'s str {
        self.string_data
    }
    
    fn get_config_data(&self) -> &'c str {
        self.config_data
    }
    
    // Method that combines data with different lifetimes
    fn format_info(&self) -> String {
        format!("Data: '{}', Config: '{}'", self.string_data, self.config_data)
    }
}

// Function that works with any lifetime
fn process_data<'a>(data: &'a str) -> (&'a str, usize) {
    let trimmed = data.trim();
    (trimmed, trimmed.len())
}

// Static lifetime example
static GLOBAL_CONFIG: &str = "production";

fn get_config() -> &'static str {
    GLOBAL_CONFIG
}

// Function that demonstrates lifetime subtyping
fn choose_str<'a, 'b: 'a>(first: &'a str, second: &'b str, use_first: bool) -> &'a str {
    if use_first {
        first
    } else {
        second  // 'b: 'a means 'b outlives 'a, so this is safe
    }
}

fn advanced_lifetimes_demo() {
    println!("\n=== Advanced Lifetimes Demo ===");
    
    // Lifetime bounds example
    let string1 = String::from("long string is long");
    let string2 = String::from("short");
    let announcement = "Comparing strings";
    
    let result = longest_with_an_announcement(
        string1.as_str(),
        string2.as_str(),
        announcement,
    );
    println!("Longest: '{}'", result);
    
    // Multiple lifetime parameters
    let data = String::from("  important data  ");
    let config = "debug";
    
    let context = Context::new(data.as_str(), config);
    println!("Context: {:?}", context);
    println!("Formatted: {}", context.format_info());
    
    // Processing with lifetimes
    let raw_data = "  Hello, Rust!  ";
    let (processed, length) = process_data(raw_data);
    println!("Processed '{}' (length: {})", processed, length);
    
    // Static lifetime
    let config = get_config();
    println!("Global config: {}", config);
    
    // Lifetime subtyping
    let long_lived = String::from("I live long");
    {
        let short_lived = String::from("I'm temporary");
        let chosen = choose_str(long_lived.as_str(), short_lived.as_str(), false);
        println!("Chosen: '{}'", chosen);
    }
    // short_lived is dropped here, but that's okay because we used it within its scope
}

advanced_lifetimes_demo();

In [ ]:
// Complex data structures with lifetimes

// Parser that holds references to input data
#[derive(Debug)]
struct Parser<'a> {
    input: &'a str,
    position: usize,
}

impl<'a> Parser<'a> {
    fn new(input: &'a str) -> Self {
        Parser { input, position: 0 }
    }
    
    fn current_char(&self) -> Option<char> {
        self.input.chars().nth(self.position)
    }
    
    fn advance(&mut self) {
        if self.position < self.input.len() {
            self.position += 1;
        }
    }
    
    fn parse_word(&mut self) -> Option<&'a str> {
        let start = self.position;
        
        // Skip whitespace
        while let Some(ch) = self.current_char() {
            if !ch.is_whitespace() {
                break;
            }
            self.advance();
        }
        
        let word_start = self.position;
        
        // Collect word characters
        while let Some(ch) = self.current_char() {
            if ch.is_whitespace() {
                break;
            }
            self.advance();
        }
        
        if word_start < self.position {
            Some(&self.input[word_start..self.position])
        } else {
            None
        }
    }
    
    fn remaining(&self) -> &'a str {
        &self.input[self.position..]
    }
}

// Iterator that yields string slices
struct WordIterator<'a> {
    parser: Parser<'a>,
}

impl<'a> WordIterator<'a> {
    fn new(input: &'a str) -> Self {
        WordIterator {
            parser: Parser::new(input),
        }
    }
}

impl<'a> Iterator for WordIterator<'a> {
    type Item = &'a str;
    
    fn next(&mut self) -> Option<Self::Item> {
        self.parser.parse_word()
    }
}

// Configuration holder with references
#[derive(Debug)]
struct Config<'a> {
    name: &'a str,
    values: Vec<&'a str>,
}

impl<'a> Config<'a> {
    fn new(name: &'a str) -> Self {
        Config {
            name,
            values: Vec::new(),
        }
    }
    
    fn add_value(&mut self, value: &'a str) {
        self.values.push(value);
    }
    
    fn get_value(&self, index: usize) -> Option<&'a str> {
        self.values.get(index).copied()
    }
    
    fn all_values(&self) -> &[&'a str] {
        &self.values
    }
}

fn lifetime_data_structures_demo() {
    println!("\n=== Lifetime-Aware Data Structures Demo ===");
    
    // Parser example
    let text = "Hello world from Rust programming";
    let mut parser = Parser::new(text);
    
    println!("Parsing text: '{}'", text);
    println!("Words found:");
    
    let mut word_count = 0;
    while let Some(word) = parser.parse_word() {
        word_count += 1;
        println!("  {}: '{}'", word_count, word);
    }
    
    // Iterator example
    let sentence = "Rust lifetimes ensure memory safety";
    let word_iter = WordIterator::new(sentence);
    
    println!("\nUsing iterator on: '{}'", sentence);
    let words: Vec<&str> = word_iter.collect();
    println!("Collected words: {:?}", words);
    
    // Configuration example
    let config_data = vec!["localhost", "8080", "debug", "true"];
    let mut config = Config::new("server");
    
    for value in &config_data {
        config.add_value(value);
    }
    
    println!("\nConfiguration: {:?}", config);
    println!("First value: {:?}", config.get_value(0));
    println!("All values: {:?}", config.all_values());
    
    // Demonstrate zero-cost abstraction
    let large_text = "The quick brown fox jumps over the lazy dog. \
                     This sentence contains every letter of the alphabet. \
                     Rust's lifetime system ensures memory safety without \
                     runtime overhead or garbage collection.";
    
    let word_count = WordIterator::new(large_text).count();
    println!("\nTotal words in large text: {}", word_count);
    
    // Show that we can still access original data
    println!("Original text length: {} characters", large_text.len());
}

lifetime_data_structures_demo();

In [ ]:
// TODO: Complete the text analyzer with proper lifetime annotations

use std::collections::HashMap;

#[derive(Debug)]
struct TextAnalyzer<'a> {
    text: &'a str,
    word_cache: Option<Vec<&'a str>>,
}

impl<'a> TextAnalyzer<'a> {
    // TODO: Create a new analyzer
    fn new(text: &'a str) -> Self {
        TextAnalyzer {
            text,
            word_cache: None,
        }
    }
    
    // TODO: Get words, caching them for efficiency
    fn words(&mut self) -> &[&'a str] {
        if self.word_cache.is_none() {
            let words: Vec<&'a str> = self.text
                .split_whitespace()
                .map(|word| word.trim_matches(|c: char| !c.is_alphabetic()))
                .filter(|word| !word.is_empty())
                .collect();
            self.word_cache = Some(words);
        }
        self.word_cache.as_ref().unwrap()
    }
    
    // TODO: Count word frequencies
    fn word_frequencies(&mut self) -> HashMap<&'a str, usize> {
        let mut frequencies = HashMap::new();
        
        for word in self.words() {
            let word_lower = word.to_lowercase();
            // Note: We need to work with the original word reference
            // In a real implementation, you might want to normalize case
            *frequencies.entry(word).or_insert(0) += 1;
        }
        
        frequencies
    }
    
    // TODO: Find the most common word
    fn most_common_word(&mut self) -> Option<(&'a str, usize)> {
        self.word_frequencies()
            .into_iter()
            .max_by_key(|(_, count)| *count)
    }
    
    // TODO: Get words longer than specified length
    fn long_words(&mut self, min_length: usize) -> Vec<&'a str> {
        self.words()
            .iter()
            .filter(|word| word.len() >= min_length)
            .copied()
            .collect()
    }
    
    // TODO: Find words that start with a specific prefix
    fn words_starting_with(&mut self, prefix: &str) -> Vec<&'a str> {
        self.words()
            .iter()
            .filter(|word| word.to_lowercase().starts_with(&prefix.to_lowercase()))
            .copied()
            .collect()
    }
    
    // TODO: Get basic statistics
    fn statistics(&mut self) -> TextStatistics<'a> {
        let words = self.words();
        let word_count = words.len();
        let char_count = self.text.chars().count();
        let avg_word_length = if word_count > 0 {
            words.iter().map(|w| w.len()).sum::<usize>() as f64 / word_count as f64
        } else {
            0.0
        };
        
        let longest_word = words.iter().max_by_key(|w| w.len()).copied();
        let shortest_word = words.iter().min_by_key(|w| w.len()).copied();
        
        TextStatistics {
            word_count,
            char_count,
            avg_word_length,
            longest_word,
            shortest_word,
        }
    }
}

#[derive(Debug)]
struct TextStatistics<'a> {
    word_count: usize,
    char_count: usize,
    avg_word_length: f64,
    longest_word: Option<&'a str>,
    shortest_word: Option<&'a str>,
}

fn text_analyzer_demo() {
    println!("\n=== Text Analyzer Demo ===");
    
    let sample_text = "Rust is a systems programming language that runs blazingly fast, \
                      prevents segfaults, and guarantees thread safety. It accomplishes \
                      these goals by being memory safe without using garbage collection. \
                      Rust is fast and memory-efficient with no runtime or garbage collector.";
    
    let mut analyzer = TextAnalyzer::new(sample_text);
    
    // Basic statistics
    let stats = analyzer.statistics();
    println!("Text Statistics:");
    println!("  Words: {}", stats.word_count);
    println!("  Characters: {}", stats.char_count);
    println!("  Average word length: {:.2}", stats.avg_word_length);
    println!("  Longest word: {:?}", stats.longest_word);
    println!("  Shortest word: {:?}", stats.shortest_word);
    
    // Most common word
    if let Some((word, count)) = analyzer.most_common_word() {
        println!("\nMost common word: '{}' (appears {} times)", word, count);
    }
    
    // Long words
    let long_words = analyzer.long_words(8);
    println!("\nWords with 8+ characters: {:?}", long_words);
    
    // Words starting with 'g'
    let g_words = analyzer.words_starting_with("g");
    println!("\nWords starting with 'g': {:?}", g_words);
    
    // Word frequencies (top 5)
    let mut frequencies: Vec<_> = analyzer.word_frequencies().into_iter().collect();
    frequencies.sort_by(|a, b| b.1.cmp(&a.1));
    
    println!("\nTop 5 word frequencies:");
    for (word, count) in frequencies.iter().take(5) {
        println!("  '{}': {}", word, count);
    }
    
    println!("\n✅ Text analysis completed using zero-copy string slices!");
}

text_analyzer_demo();

In [ ]:
// Advanced associated types and type families

// Iterator trait with associated types
trait Iterator {
    type Item;
    
    fn next(&mut self) -> Option<Self::Item>;
    
    // Default implementations using associated types
    fn collect<C: FromIterator<Self::Item>>(self) -> C
    where
        Self: Sized,
    {
        FromIterator::from_iter(self)
    }
    
    fn map<B, F>(self, f: F) -> Map<Self, F>
    where
        Self: Sized,
        F: FnMut(Self::Item) -> B,
    {
        Map { iter: self, f }
    }
}

// Map iterator adapter
struct Map<I, F> {
    iter: I,
    f: F,
}

impl<B, I: Iterator, F> Iterator for Map<I, F>
where
    F: FnMut(I::Item) -> B,
{
    type Item = B;
    
    fn next(&mut self) -> Option<Self::Item> {
        self.iter.next().map(&mut self.f)
    }
}

// FromIterator trait
trait FromIterator<A> {
    fn from_iter<T: Iterator<Item = A>>(iter: T) -> Self;
}

impl<T> FromIterator<T> for Vec<T> {
    fn from_iter<I: Iterator<Item = T>>(iter: I) -> Self {
        let mut vec = Vec::new();
        let mut iter = iter;
        while let Some(item) = iter.next() {
            vec.push(item);
        }
        vec
    }
}

// Custom iterator implementation
struct Range {
    current: i32,
    end: i32,
}

impl Range {
    fn new(start: i32, end: i32) -> Self {
        Range { current: start, end }
    }
}

impl Iterator for Range {
    type Item = i32;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.current < self.end {
            let current = self.current;
            self.current += 1;
            Some(current)
        } else {
            None
        }
    }
}

// Graph trait with associated types for type families
trait Graph {
    type Node;
    type Edge;
    type NodeIter: Iterator<Item = Self::Node>;
    type EdgeIter: Iterator<Item = Self::Edge>;
    
    fn nodes(&self) -> Self::NodeIter;
    fn edges(&self) -> Self::EdgeIter;
    fn neighbors(&self, node: &Self::Node) -> Self::NodeIter;
}

// Simple graph implementation
struct SimpleGraph {
    nodes: Vec<u32>,
    edges: Vec<(u32, u32)>,
}

impl SimpleGraph {
    fn new() -> Self {
        SimpleGraph {
            nodes: Vec::new(),
            edges: Vec::new(),
        }
    }
    
    fn add_node(&mut self, node: u32) {
        if !self.nodes.contains(&node) {
            self.nodes.push(node);
        }
    }
    
    fn add_edge(&mut self, from: u32, to: u32) {
        self.add_node(from);
        self.add_node(to);
        self.edges.push((from, to));
    }
}

impl Graph for SimpleGraph {
    type Node = u32;
    type Edge = (u32, u32);
    type NodeIter = std::vec::IntoIter<u32>;
    type EdgeIter = std::vec::IntoIter<(u32, u32)>;
    
    fn nodes(&self) -> Self::NodeIter {
        self.nodes.clone().into_iter()
    }
    
    fn edges(&self) -> Self::EdgeIter {
        self.edges.clone().into_iter()
    }
    
    fn neighbors(&self, node: &Self::Node) -> Self::NodeIter {
        let neighbors: Vec<u32> = self.edges
            .iter()
            .filter_map(|(from, to)| {
                if from == node {
                    Some(*to)
                } else {
                    None
                }
            })
            .collect();
        neighbors.into_iter()
    }
}

// Generic algorithm using associated types
fn graph_analysis<G: Graph>(graph: &G) 
where
    G::Node: std::fmt::Debug + Clone,
    G::Edge: std::fmt::Debug,
{
    println!("Graph Analysis:");
    
    let nodes: Vec<G::Node> = graph.nodes().collect();
    println!("  Nodes: {:?}", nodes);
    
    let edges: Vec<G::Edge> = graph.edges().collect();
    println!("  Edges: {:?}", edges);
    
    if let Some(first_node) = nodes.first() {
        let neighbors: Vec<G::Node> = graph.neighbors(first_node).collect();
        println!("  Neighbors of {:?}: {:?}", first_node, neighbors);
    }
}

fn associated_types_demo() {
    println!("=== Associated Types and Type Families ===");
    
    // Custom iterator usage
    let range = Range::new(1, 6);
    let doubled: Vec<i32> = range.map(|x| x * 2).collect();
    println!("Doubled range: {:?}", doubled);
    
    // Graph example
    let mut graph = SimpleGraph::new();
    graph.add_edge(1, 2);
    graph.add_edge(1, 3);
    graph.add_edge(2, 4);
    graph.add_edge(3, 4);
    
    graph_analysis(&graph);
}

associated_types_demo();

In [ ]:
// Advanced trait objects and dynamic dispatch patterns

// Object-safe trait for drawing
trait Drawable {
    fn draw(&self);
    fn area(&self) -> f64;
    
    // Object-safe method with default implementation
    fn describe(&self) {
        println!("Shape with area: {:.2}", self.area());
    }
}

// Concrete implementations
struct Circle {
    radius: f64,
}

impl Circle {
    fn new(radius: f64) -> Self {
        Circle { radius }
    }
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing circle with radius {}", self.radius);
    }
    
    fn area(&self) -> f64 {
        std::f64::consts::PI * self.radius * self.radius
    }
}

struct Rectangle {
    width: f64,
    height: f64,
}

impl Rectangle {
    fn new(width: f64, height: f64) -> Self {
        Rectangle { width, height }
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
        println!("Drawing rectangle {}x{}", self.width, self.height);
    }
    
    fn area(&self) -> f64 {
        self.width * self.height
    }
}

// Trait for cloning trait objects
trait CloneableDrawable: Drawable {
    fn clone_box(&self) -> Box<dyn CloneableDrawable>;
}

impl<T> CloneableDrawable for T
where
    T: Drawable + Clone + 'static,
{
    fn clone_box(&self) -> Box<dyn CloneableDrawable> {
        Box::new(self.clone())
    }
}

#[derive(Clone)]
struct Triangle {
    base: f64,
    height: f64,
}

impl Triangle {
    fn new(base: f64, height: f64) -> Self {
        Triangle { base, height }
    }
}

impl Drawable for Triangle {
    fn draw(&self) {
        println!("Drawing triangle with base {} and height {}", self.base, self.height);
    }
    
    fn area(&self) -> f64 {
        0.5 * self.base * self.height
    }
}

// Canvas that works with trait objects
struct Canvas {
    shapes: Vec<Box<dyn Drawable>>,
}

impl Canvas {
    fn new() -> Self {
        Canvas {
            shapes: Vec::new(),
        }
    }
    
    fn add_shape(&mut self, shape: Box<dyn Drawable>) {
        self.shapes.push(shape);
    }
    
    fn draw_all(&self) {
        println!("Drawing all shapes:");
        for (i, shape) in self.shapes.iter().enumerate() {
            print!("  {}: ", i + 1);
            shape.draw();
        }
    }
    
    fn total_area(&self) -> f64 {
        self.shapes.iter().map(|shape| shape.area()).sum()
    }
    
    fn describe_all(&self) {
        println!("Shape descriptions:");
        for (i, shape) in self.shapes.iter().enumerate() {
            print!("  {}: ", i + 1);
            shape.describe();
        }
    }
}

// Function that works with any drawable
fn process_drawable(drawable: &dyn Drawable) {
    drawable.draw();
    println!("  Area: {:.2}", drawable.area());
}

// Function that returns trait objects
fn create_shape(shape_type: &str, size: f64) -> Box<dyn Drawable> {
    match shape_type {
        "circle" => Box::new(Circle::new(size)),
        "square" => Box::new(Rectangle::new(size, size)),
        "triangle" => Box::new(Triangle::new(size, size)),
        _ => Box::new(Circle::new(1.0)), // Default
    }
}

// Advanced: Trait object with multiple traits
trait Named {
    fn name(&self) -> &str;
}

impl Named for Circle {
    fn name(&self) -> &str {
        "Circle"
    }
}

impl Named for Rectangle {
    fn name(&self) -> &str {
        "Rectangle"
    }
}

impl Named for Triangle {
    fn name(&self) -> &str {
        "Triangle"
    }
}

// Function using multiple trait bounds
fn process_named_drawable(item: &(dyn Drawable + Named)) {
    println!("Processing {}", item.name());
    item.draw();
    println!("  Area: {:.2}", item.area());
}

fn trait_objects_demo() {
    println!("\n=== Trait Objects and Dynamic Dispatch ===");
    
    let mut canvas = Canvas::new();
    
    // Add different shapes as trait objects
    canvas.add_shape(Box::new(Circle::new(5.0)));
    canvas.add_shape(Box::new(Rectangle::new(4.0, 6.0)));
    canvas.add_shape(Box::new(Triangle::new(3.0, 8.0)));
    
    // Use dynamic dispatch
    canvas.draw_all();
    println!("\nTotal area: {:.2}", canvas.total_area());
    
    canvas.describe_all();
    
    // Create shapes dynamically
    println!("\nDynamically created shapes:");
    let shapes = vec![
        create_shape("circle", 3.0),
        create_shape("square", 4.0),
        create_shape("triangle", 5.0),
    ];
    
    for shape in &shapes {
        process_drawable(shape.as_ref());
        println!();
    }
    
    // Multiple trait objects
    println!("Multiple trait bounds:");
    let circle = Circle::new(2.0);
    let rectangle = Rectangle::new(3.0, 4.0);
    
    process_named_drawable(&circle);
    println!();
    process_named_drawable(&rectangle);
}

trait_objects_demo();

In [ ]:
// Higher-ranked trait bounds and advanced closure patterns

// Function that takes a closure working with any lifetime
fn apply_to_string<F>(f: F, s: &str) -> String
where
    F: for<'a> Fn(&'a str) -> String,
{
    f(s)
}

// Function that works with references of any lifetime
fn process_with_closure<F, T>(items: &[T], mut f: F) -> Vec<String>
where
    F: for<'a> FnMut(&'a T) -> String,
    T: std::fmt::Debug,
{
    items.iter().map(|item| f(item)).collect()
}

// Trait with higher-ranked bounds
trait Processor {
    fn process<F>(&self, f: F) -> String
    where
        F: for<'a> Fn(&'a str) -> &'a str;
}

struct TextProcessor {
    text: String,
}

impl TextProcessor {
    fn new(text: String) -> Self {
        TextProcessor { text }
    }
}

impl Processor for TextProcessor {
    fn process<F>(&self, f: F) -> String
    where
        F: for<'a> Fn(&'a str) -> &'a str,
    {
        f(&self.text).to_string()
    }
}

// Advanced closure trait usage
struct EventHandler<F> {
    handler: F,
}

impl<F> EventHandler<F>
where
    F: Fn(&str) -> bool,
{
    fn new(handler: F) -> Self {
        EventHandler { handler }
    }
    
    fn handle_event(&self, event: &str) -> bool {
        (self.handler)(event)
    }
}

// Function that demonstrates different closure traits
fn closure_traits_demo() {
    println!("\n=== Higher-Ranked Trait Bounds and Closures ===");
    
    // Higher-ranked trait bounds with closures
    let result = apply_to_string(|s| s.to_uppercase(), "hello world");
    println!("Applied to string: {}", result);
    
    let result = apply_to_string(|s| format!("[{}]", s), "rust");
    println!("Applied to string: {}", result);
    
    // Process with closure
    let numbers = vec![1, 2, 3, 4, 5];
    let results = process_with_closure(&numbers, |n| format!("Number: {}", n));
    println!("Processed numbers: {:?}", results);
    
    // Processor trait
    let processor = TextProcessor::new("Hello, Rust!".to_string());
    let result = processor.process(|s| s.trim());
    println!("Processed text: '{}'", result);
    
    // Event handler with different closure types
    let handler1 = EventHandler::new(|event| {
        println!("Handling event: {}", event);
        event.len() > 5
    });
    
    let events = vec!["click", "double_click", "hover", "key_press"];
    
    println!("\nEvent handling results:");
    for event in events {
        let handled = handler1.handle_event(event);
        println!("  Event '{}' handled: {}", event, handled);
    }
    
    // Demonstrate different closure traits
    demonstrate_closure_traits();
}

fn demonstrate_closure_traits() {
    println!("\nClosure trait demonstrations:");
    
    // Fn: can be called multiple times, borrows immutably
    let multiplier = 2;
    let multiply_fn = |x: i32| x * multiplier;
    
    fn use_fn<F>(f: F, values: &[i32]) -> Vec<i32>
    where
        F: Fn(i32) -> i32,
    {
        values.iter().map(|&x| f(x)).collect()
    }
    
    let numbers = vec![1, 2, 3, 4, 5];
    let multiplied = use_fn(multiply_fn, &numbers);
    println!("  Fn trait result: {:?}", multiplied);
    
    // FnMut: can be called multiple times, borrows mutably
    let mut counter = 0;
    let mut count_calls = |x: i32| {
        counter += 1;
        x + counter
    };
    
    fn use_fn_mut<F>(mut f: F, values: &[i32]) -> Vec<i32>
    where
        F: FnMut(i32) -> i32,
    {
        values.iter().map(|&x| f(x)).collect()
    }
    
    let counted = use_fn_mut(count_calls, &numbers);
    println!("  FnMut trait result: {:?}", counted);
    
    // FnOnce: can be called once, takes ownership
    let owned_string = String::from("Hello");
    let consume_string = |suffix: &str| {
        format!("{} {}", owned_string, suffix)
    };
    
    fn use_fn_once<F>(f: F, suffix: &str) -> String
    where
        F: FnOnce(&str) -> String,
    {
        f(suffix)
    }
    
    let result = use_fn_once(consume_string, "World");
    println!("  FnOnce trait result: {}", result);
    // Note: owned_string and consume_string are no longer available
}

closure_traits_demo();

In [ ]:
// TODO: Complete the plugin system using advanced trait patterns

use std::collections::HashMap;
use std::any::{Any, TypeId};

// Plugin trait with associated types
trait Plugin: Any + Send + Sync {
    type Config;
    type Output;
    
    fn name(&self) -> &str;
    fn version(&self) -> &str;
    fn execute(&self, config: &Self::Config) -> Result<Self::Output, String>;
    
    // Default implementation
    fn description(&self) -> String {
        format!("{} v{}", self.name(), self.version())
    }
}

// Trait for plugins that can be configured dynamically
trait ConfigurablePlugin: Plugin {
    fn configure(&mut self, key: &str, value: &str) -> Result<(), String>;
    fn get_config(&self) -> HashMap<String, String>;
}

// Trait for plugins that can process data streams
trait StreamProcessor: Plugin {
    type Item;
    
    fn process_stream<I>(&self, input: I) -> Box<dyn Iterator<Item = Self::Item>>
    where
        I: Iterator<Item = Self::Item> + 'static;
}

// Text processing plugin
#[derive(Debug)]
struct TextProcessorPlugin {
    config: HashMap<String, String>,
}

impl TextProcessorPlugin {
    fn new() -> Self {
        let mut config = HashMap::new();
        config.insert("case".to_string(), "lower".to_string());
        config.insert("trim".to_string(), "true".to_string());
        
        TextProcessorPlugin { config }
    }
}

#[derive(Debug)]
struct TextConfig {
    to_uppercase: bool,
    trim_whitespace: bool,
    prefix: Option<String>,
}

impl Plugin for TextProcessorPlugin {
    type Config = TextConfig;
    type Output = String;
    
    fn name(&self) -> &str {
        "TextProcessor"
    }
    
    fn version(&self) -> &str {
        "1.0.0"
    }
    
    fn execute(&self, config: &Self::Config) -> Result<Self::Output, String> {
        let mut result = "Sample text processing".to_string();
        
        if config.trim_whitespace {
            result = result.trim().to_string();
        }
        
        if config.to_uppercase {
            result = result.to_uppercase();
        }
        
        if let Some(prefix) = &config.prefix {
            result = format!("{}: {}", prefix, result);
        }
        
        Ok(result)
    }
}

impl ConfigurablePlugin for TextProcessorPlugin {
    fn configure(&mut self, key: &str, value: &str) -> Result<(), String> {
        match key {
            "case" | "trim" => {
                self.config.insert(key.to_string(), value.to_string());
                Ok(())
            }
            _ => Err(format!("Unknown configuration key: {}", key)),
        }
    }
    
    fn get_config(&self) -> HashMap<String, String> {
        self.config.clone()
    }
}

impl StreamProcessor for TextProcessorPlugin {
    type Item = String;
    
    fn process_stream<I>(&self, input: I) -> Box<dyn Iterator<Item = Self::Item>>
    where
        I: Iterator<Item = Self::Item> + 'static,
    {
        let case_setting = self.config.get("case").cloned().unwrap_or_else(|| "lower".to_string());
        let trim_setting = self.config.get("trim").cloned().unwrap_or_else(|| "true".to_string());
        
        let should_uppercase = case_setting == "upper";
        let should_trim = trim_setting == "true";
        
        Box::new(input.map(move |mut text| {
            if should_trim {
                text = text.trim().to_string();
            }
            if should_uppercase {
                text = text.to_uppercase();
            } else {
                text = text.to_lowercase();
            }
            text
        }))
    }
}

// Number processing plugin
#[derive(Debug)]
struct NumberProcessorPlugin {
    multiplier: f64,
}

impl NumberProcessorPlugin {
    fn new(multiplier: f64) -> Self {
        NumberProcessorPlugin { multiplier }
    }
}

#[derive(Debug)]
struct NumberConfig {
    operation: String,
    value: f64,
}

impl Plugin for NumberProcessorPlugin {
    type Config = NumberConfig;
    type Output = f64;
    
    fn name(&self) -> &str {
        "NumberProcessor"
    }
    
    fn version(&self) -> &str {
        "2.1.0"
    }
    
    fn execute(&self, config: &Self::Config) -> Result<Self::Output, String> {
        let base_value = 42.0;
        
        let result = match config.operation.as_str() {
            "add" => base_value + config.value,
            "multiply" => base_value * config.value,
            "divide" => {
                if config.value == 0.0 {
                    return Err("Division by zero".to_string());
                }
                base_value / config.value
            }
            _ => return Err(format!("Unknown operation: {}", config.operation)),
        };
        
        Ok(result * self.multiplier)
    }
}

impl StreamProcessor for NumberProcessorPlugin {
    type Item = f64;
    
    fn process_stream<I>(&self, input: I) -> Box<dyn Iterator<Item = Self::Item>>
    where
        I: Iterator<Item = Self::Item> + 'static,
    {
        let multiplier = self.multiplier;
        Box::new(input.map(move |x| x * multiplier))
    }
}

// Plugin registry using trait objects
struct PluginRegistry {
    plugins: HashMap<String, Box<dyn Plugin<Config = (), Output = String>>>,
    configurable_plugins: HashMap<String, Box<dyn ConfigurablePlugin<Config = (), Output = String>>>,
}

impl PluginRegistry {
    fn new() -> Self {
        PluginRegistry {
            plugins: HashMap::new(),
            configurable_plugins: HashMap::new(),
        }
    }
    
    fn list_plugins(&self) {
        println!("Registered plugins:");
        for (name, plugin) in &self.plugins {
            println!("  - {}: {}", name, plugin.description());
        }
        
        for (name, plugin) in &self.configurable_plugins {
            println!("  - {} (configurable): {}", name, plugin.description());
        }
    }
}

// Generic plugin executor
fn execute_plugin<P, C>(plugin: &P, config: &C) -> Result<P::Output, String>
where
    P: Plugin<Config = C>,
{
    println!("Executing plugin: {}", plugin.description());
    plugin.execute(config)
}

// Higher-order function for plugin processing
fn process_with_plugins<F, T>(plugins: Vec<Box<dyn StreamProcessor<Item = T>>>, 
                             data: Vec<T>, 
                             combiner: F) -> Vec<T>
where
    F: Fn(Vec<T>) -> Vec<T>,
    T: Clone + 'static,
{
    let mut results = Vec::new();
    
    for plugin in plugins {
        let processed: Vec<T> = plugin.process_stream(data.clone().into_iter()).collect();
        results.extend(processed);
    }
    
    combiner(results)
}

fn plugin_system_demo() {
    println!("\n=== Advanced Plugin System Demo ===");
    
    // Create plugins
    let mut text_plugin = TextProcessorPlugin::new();
    let number_plugin = NumberProcessorPlugin::new(2.0);
    
    // Configure text plugin
    text_plugin.configure("case", "upper").unwrap();
    text_plugin.configure("trim", "true").unwrap();
    
    println!("Text plugin config: {:?}", text_plugin.get_config());
    
    // Execute plugins with different configs
    let text_config = TextConfig {
        to_uppercase: true,
        trim_whitespace: true,
        prefix: Some("PROCESSED".to_string()),
    };
    
    let text_result = execute_plugin(&text_plugin, &text_config).unwrap();
    println!("Text processing result: {}", text_result);
    
    let number_config = NumberConfig {
        operation: "multiply".to_string(),
        value: 3.0,
    };
    
    let number_result = execute_plugin(&number_plugin, &number_config).unwrap();
    println!("Number processing result: {:.2}", number_result);
    
    // Stream processing
    println!("\nStream processing:");
    let text_data = vec![
        "  Hello World  ".to_string(),
        "  Rust Programming  ".to_string(),
        "  Advanced Traits  ".to_string(),
    ];
    
    let processed_text: Vec<String> = text_plugin
        .process_stream(text_data.into_iter())
        .collect();
    
    println!("Processed text stream: {:?}", processed_text);
    
    let number_data = vec![1.0, 2.0, 3.0, 4.0, 5.0];
    let processed_numbers: Vec<f64> = number_plugin
        .process_stream(number_data.into_iter())
        .collect();
    
    println!("Processed number stream: {:?}", processed_numbers);
    
    // Plugin registry demonstration
    let registry = PluginRegistry::new();
    registry.list_plugins();
    
    println!("\nPlugin system demonstration complete!");
}

plugin_system_demo();

---

## ⚠️ Common Pitfalls: Lifetimes & Advanced Traits

### 1. Lifetime Annotation Confusion
```rust
// ❌ Wrong: trying to return reference to local variable
fn dangle<'a>() -> &'a str {
    let s = String::from("hello");
    &s  // Error: s is dropped at end of function
}
```
**Solution:** Return owned data or take reference as parameter:
```rust
fn no_dangle() -> String {
    String::from("hello")  // Transfer ownership
}
```

### 2. Struct Lifetime Outliving Referenced Data
```rust
struct Holder<'a> {
    data: &'a str,
}

let holder;
{
    let s = String::from("hello");
    holder = Holder { data: &s };  // ❌ Error: s doesn't live long enough
}
println!("{}", holder.data);
```
**Solution:** Ensure referenced data outlives the struct

### 3. Multiple Lifetime Parameters Confusion
```rust
// ❌ Wrong: trying to return reference with wrong lifetime
fn wrong<'a, 'b>(x: &'a str, y: &'b str) -> &'a str {
    if x.len() > y.len() { x } else { y }  // Error: y has lifetime 'b, not 'a
}
```
**Solution:** Use same lifetime when return could be either:
```rust
fn correct<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() { x } else { y }
}
```

### 4. Static Lifetime Misuse
```rust
// ⚠️ Overly restrictive: forces 'static lifetime
fn process(s: &'static str) -> &'static str {
    s
}
// Can only call with string literals, not runtime strings
```
**Solution:** Use generic lifetime unless truly need 'static:
```rust
fn process<'a>(s: &'a str) -> &'a str {
    s
}
```

### 5. Associated Type vs Generic Parameter
```rust
// When to use associated type?
trait Container<T> {  // ⚠️ Generic: can implement multiple times for different T
    fn add(&mut self, item: T);
}

trait Container2 {  // ✅ Associated type: only one implementation per type
    type Item;
    fn add(&mut self, item: Self::Item);
}
```
**Rule:** Use associated types when there's one logical type per implementation

### 6. Higher-Ranked Trait Bounds (HRTB) Complexity
```rust
// ❌ Error: lifetime parameter not in scope
fn apply<F>(f: F) where F: Fn(&str) -> &str {
    // Missing lifetime relationship
}
```
**Solution:** Use HRTB with `for<'a>`:
```rust
fn apply<F>(f: F) where F: for<'a> Fn(&'a str) -> &'a str {
    // Works for any lifetime 'a
}
```

### 7. Trait Object Lifetime Defaults
```rust
// These are equivalent:
Box<dyn Trait>           // Defaults to 'static
Box<dyn Trait + 'static>

// To allow non-static references:
Box<dyn Trait + 'a>  // Explicit lifetime
```
**Pitfall:** Forgetting that `Box<dyn Trait>` requires 'static by default

### 8. Lifetime Elision Misunderstanding
```rust
// These are equivalent (elision rules):
fn first(s: &str) -> &str { }  // Elided
fn first<'a>(s: &'a str) -> &'a str { }  // Explicit

// But this needs explicit annotation:
fn longest(x: &str, y: &str) -> &str { }  // ❌ Error: ambiguous
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str { }  // ✅ Explicit
```

**📚 Rust Book References:**
- [Chapter 10.3 - Validating References with Lifetimes](https://doc.rust-lang.org/book/ch10-03-lifetime-syntax.html)
- [Chapter 19.2 - Advanced Traits](https://doc.rust-lang.org/book/ch19-03-advanced-traits.html)